In [63]:
import os.path
import numpy as np
raw_documents = []
documents=[]
snippets = []
import string
remove = str.maketrans('','',string.punctuation) 
with open( os.path.join("data", "articles.txt") ,"r") as fin:
    for line in fin.readlines():
        words=line.lower().translate(remove).split(" ")
        documents.append(words)
print("Read %d raw text documents" % len(documents))
print (documents[0][0])

Read 4551 raw text documents
barclays


In [64]:
custom_stop_words = []
with open( "stopwords.txt", "r" ) as fin:
    for line in fin.readlines():
        custom_stop_words.append( line.strip() )
# note that we need to make it hashable
print("Stopword list has %d entries" % len(custom_stop_words) )

Stopword list has 350 entries


In [65]:
vocabulary={}
for j in range(500):
    document=documents[j]
    vocabulary[j]=[]
    for i in range(len(document)):
        if document[i] not in custom_stop_words:
            if len(document[i])>=2:
                vocabulary[j].append(document[i]) 

print (vocabulary[0])

['barclays', 'defiance', 'fines', 'merit', 'barclays', 'disgraced', 'ways', 'prefinancial', 'crisis', 'boom', 'tempting', 'think', 'bank', 'asked', 'department', 'justice', 'pay', 'large', 'bill', 'polluting', 'financial', 'system', 'mortgage', 'junk', '2005', '2007', 'cough', 'apologise', 'learn', 'humility', 'view', 'chief', 'executive', 'jes', 'staley', 'barclays', 'thinks', 'doj’s', 'claims', '“disconnected', 'facts”', '“an', 'obligation', 'shareholders', 'customers', 'clients', 'employees', 'defend', 'against', 'unreasonable', 'allegations', 'demands”', 'stance', 'possibly', 'foolhardy', 'going', 'open', 'legal', 'battle', 'powerful', 'prosecutor', 'risky', 'especially', 'losing', 'grudging', 'respect', 'staley', 'barclays', 'order', 'system', 'dishing', 'fines', 'errant', 'banks', 'mortgage', 'sins', 'resemble', 'casino', 'approach', 'prefers', 'settlements', 'behind', 'closed', 'doors', 'difference', 'size', 'penalties', 'explained', 'occasional', 'leaks', 'negotiating', 'demand

In [66]:
word2idx={}
idx2word={}
i=0
word_count={}

for k in range(len(vocabulary)):
    document=vocabulary[k]
    for word in document:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1


datas={}
count={}
for j in range(len(vocabulary)):
    document=vocabulary[j]
    datas[j]=[]
    for w in document:
        if word_count[w]>20:
            count[w]=word_count[w]
            if w in word2idx:
                datas[j].append(word2idx[w])
                continue
            word2idx[w]=i
            idx2word[i]=w
            datas[j].append(word2idx[w])
            i=i+1


In [67]:
print (len(count))
print (len(word2idx))
print (len(idx2word))
print (len(count))



2314
2314
2314
2314


In [68]:
all_data=[]
for i in range(len(datas)):
    data=datas[i]
    for idx in data:
        all_data.append(idx)
    
print (len(all_data)) 

148671


In [69]:
word_count=np.array(list(count.values()))
print (len(word_count))
neg_weight= np.power(word_count, 3/4)
neg_weight = neg_weight / neg_weight.sum()
print (neg_weight)

2314
[0.00042066 0.00022671 0.00043716 ... 0.00019892 0.00024683 0.00019892]


In [70]:
from torch import nn
import torch
class Model(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(Model, self).__init__()
        self.input_emb = nn.Embedding(vocab_size, emb_dim)
        self.output_emb = nn.Embedding(vocab_size, emb_dim)
        self.log_sigmoid = nn.LogSigmoid()
        initrange = (2.0 / (vocab_size + emb_dim)) ** 0.5  # Xavier init
        self.input_emb.weight.data.uniform_(-initrange, initrange)
        self.output_emb.weight.data.uniform_(-0, 0)


    def forward(self, pos_u, pos_v, neg_v):
        # u,v: [batch_size, emb_dim]
        pos_u = self.input_emb(pos_u)
        pos_v = self.output_emb(pos_v)
        # positive_val: [batch_size]
        positive_val=self.log_sigmoid(torch.sum(pos_u *  pos_v, dim=1))
        # neg_v: [batch_size, neg_size, emb_dim]
        neg_v= self.output_emb(neg_v)
        # [batch_size, neg_size, emb_dim] x [batch_size, emb_dim, 1] = [batch_size, neg_size, 1]
        # neg_vals: [batch_size, neg_size]
        neg_vals = -torch.bmm(neg_v, pos_u.unsqueeze(2)).squeeze(2)
        # neg_val: [batch_size]
        neg_val=self.log_sigmoid(neg_vals)
        neg_val=torch.sum(neg_val,dim=1)
    
        loss = positive_val + neg_val
        return -loss.mean()

    def predict(self, inputs):
        return self.input_emb(inputs)
    def get_embeddings(self):
        return self.input_emb.weight.data.numpy()

In [71]:
model=Model(len(word2idx),2)
inputs=torch.tensor([2,3])
context=torch.tensor([0,1])
neg=torch.tensor([[4,5,9],[6,7,9]])
model(inputs,context,neg)

tensor(2.7726, grad_fn=<NegBackward>)

In [72]:
import collections
class DataLoader(object):
    def __init__(self, data,process=True):
        self.data=data
        self.process=process
        
    def set_data(self,data):
        self.data=data
    def generate_batch(self, window_size, batch_size):
        data = self.data
        global data_index
        span = 2 * window_size + 1
        context = np.ndarray(shape=(batch_size,2 * window_size), dtype=np.int64)
        labels = np.ndarray(shape=(batch_size), dtype=np.int64)
        pos_pair = [] 
        buffer = collections.deque(maxlen=span)
        pos_u2context={}
        if data_index + span > len(data): 
            data_index = 0 
            self.process=False
        buffer = data[data_index:data_index + span]
        pos_u = []
        pos_v = [] 
        context[0,:] = buffer[:window_size]+buffer[window_size+1:]
        
        for i in range(batch_size):
            data_index += 1
            context[i,:] = buffer[:window_size]+buffer[window_size+1:]
            labels[i] = buffer[window_size]
            if data_index + span > len(data):
                buffer[:] = data[:span]
                data_index = 0
                self.process=False

            else:
                buffer = data[data_index:data_index + span]

            for j in range(span-1):
                pos_u.append(labels[i])
                pos_v.append(context[i,j])
            pos_u2context[labels[i]]=context[i,:]
        return np.array(pos_u), np.array(pos_v), pos_u2context
    
    def get_neg(self, pos_u, num_neg, context):
        neg_samples =[]
        for i in pos_u:
            Flag=True
            while Flag:
                neg_sample=np.random.choice(len(neg_weight), num_neg, p=neg_weight)
                for pos in context[i]:
                    if pos in neg_sample:
                        Flag=True
                        break;
                    Flag=False
                if Flag==False:
                    neg_samples.append(neg_sample)
        return neg_samples

In [73]:
data_index=0
#测试dataloader
data_index=0
print (datas[0])
dataLoader=DataLoader(datas[0])
pos_u, pos_v, contex=dataLoader.generate_batch(2,10)
print (pos_u)
print (pos_v)
neg_samples=dataLoader.get_neg(pos_u,2,contex)

[0, 1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 0, 37, 13, 1, 38, 14, 39, 40, 41, 42, 43, 44, 45, 46, 5, 47, 6, 48, 49, 50, 51, 52, 25, 38, 53, 53, 5, 54, 55, 49, 56, 51, 0, 57, 58, 59, 14, 5, 60, 61, 62, 0, 63, 64, 65, 0, 6, 66, 67, 68, 69, 0, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 0, 23, 82, 83, 84, 85, 86, 87, 88, 43, 89, 0, 19, 90, 5, 91, 92, 79, 93, 94, 95, 96]
[ 0  0  0  0  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6  6  6  6
  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10]
[ 0  1  2  3  1  0  3  4  0  2  4  5  2  3  5  6  3  4  6  7  4  5  7  8
  5  6  8  9  6  7  9 10  7  8 10 11  8  9 11 12]


In [74]:
#dmodel
import time
from torch.autograd import Variable
class DCWTE:
    def __init__(self, datas, word_size ,embedding_dim=100, epoch_num=200, batch_size=100, windows_size=5, neg_sample_num=5, learning_ratio=0.01):
        self.datas=datas
        self.embedding_dim = embedding_dim
        self.windows_size = windows_size
        self.word_size = word_size
        self.batch_size = batch_size
        self.epoch_num = epoch_num
        self.neg_sample_num = neg_sample_num
        self.model=Model(self.word_size, self.embedding_dim) 
        self.k=0
        self.learning_ratio=learning_ratio
    def test(self):
        embeddings=self.model.get_embeddings()
        return embeddings
    def train(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_ratio)
        if torch.cuda.is_available():
            self.model.cuda()
        for epoch in range(self.epoch_num):
            batch_num = 0
            self.k+=1
            print (epoch)
            start = time.time()     
            for i in range(len(self.datas)):
                data=datas[i]
                self.dataloader=DataLoader(data)
                self.dataloader.process = True
                batch_num = 0
                
                while self.dataloader.process:
                
                    pos_u, pos_v, pos_u2context= self.dataloader.generate_batch(self.windows_size, self.batch_size)
                    neg_v=self.dataloader.get_neg(pos_u,self.neg_sample_num,pos_u2context)

                    pos_u = Variable(torch.LongTensor(pos_u))
                    pos_v = Variable(torch.LongTensor(pos_v))
                    neg_v = Variable(torch.LongTensor(neg_v))
                    if torch.cuda.is_available():
                        pos_u = pos_u.cuda()
                        pos_v = pos_v.cuda()
                        neg_v = neg_v.cuda()

                    optimizer.zero_grad()
                    loss = self.model(pos_u, pos_v, neg_v)
                    batch_num = batch_num + 1 
                    loss.backward()

                    optimizer.step()
                    
                print( "epho : " + str(epoch)  + " doc"+str(i)+" batch_num:"+str(batch_num)+" loss: "+str(loss.item()) + "\n")
               
                
                
                
                

In [75]:
print (neg_weight)
print (len(all_data))
dcwte=DCWTE(datas,len(word2idx))
k=0

[0.00042066 0.00022671 0.00043716 ... 0.00019892 0.00024683 0.00019892]
148671


In [91]:
dcwte.batch_size=200
dcwte.windows_size=2
dcwte.neg_sample_num=5
dcwte.epoch_num=1000
dcwte.learning_ratio=0.001
dcwte.embedding_dim=200
print (dcwte.k)
print (dcwte.learning_ratio)

9
0.001


In [90]:
dcwte.train()

0
epho : 0 doc0 batch_num:1 loss: 2.6059370040893555

epho : 0 doc1 batch_num:1 loss: 2.3455655574798584

epho : 0 doc2 batch_num:1 loss: 2.6355929374694824

epho : 0 doc3 batch_num:2 loss: 2.393799304962158

epho : 0 doc4 batch_num:2 loss: 2.4590628147125244

epho : 0 doc5 batch_num:1 loss: 2.504417896270752

epho : 0 doc6 batch_num:1 loss: 2.5083348751068115

epho : 0 doc7 batch_num:2 loss: 2.506185531616211

epho : 0 doc8 batch_num:1 loss: 2.5532774925231934

epho : 0 doc9 batch_num:1 loss: 2.509350299835205

epho : 0 doc10 batch_num:7 loss: 2.4271297454833984

epho : 0 doc11 batch_num:1 loss: 2.514957904815674

epho : 0 doc12 batch_num:2 loss: 2.6262152194976807

epho : 0 doc13 batch_num:1 loss: 2.5517632961273193

epho : 0 doc14 batch_num:2 loss: 2.716719150543213

epho : 0 doc15 batch_num:6 loss: 2.467057466506958

epho : 0 doc16 batch_num:2 loss: 2.3854763507843018

epho : 0 doc17 batch_num:1 loss: 2.5423853397369385

epho : 0 doc18 batch_num:1 loss: 2.490832567214966

epho : 0 

epho : 0 doc157 batch_num:2 loss: 2.6118602752685547

epho : 0 doc158 batch_num:1 loss: 2.576317071914673

epho : 0 doc159 batch_num:1 loss: 2.6030919551849365

epho : 0 doc160 batch_num:2 loss: 2.5830466747283936

epho : 0 doc161 batch_num:1 loss: 2.588695764541626

epho : 0 doc162 batch_num:1 loss: 2.409390926361084

epho : 0 doc163 batch_num:1 loss: 2.0901830196380615

epho : 0 doc164 batch_num:1 loss: 2.5585715770721436

epho : 0 doc165 batch_num:1 loss: 2.4489917755126953

epho : 0 doc166 batch_num:1 loss: 2.5020787715911865

epho : 0 doc167 batch_num:2 loss: 2.5361928939819336

epho : 0 doc168 batch_num:1 loss: 2.582474708557129

epho : 0 doc169 batch_num:2 loss: 2.4805638790130615

epho : 0 doc170 batch_num:1 loss: 2.5831007957458496

epho : 0 doc171 batch_num:1 loss: 2.5808942317962646

epho : 0 doc172 batch_num:1 loss: 2.523197650909424

epho : 0 doc173 batch_num:1 loss: 2.4637644290924072

epho : 0 doc174 batch_num:1 loss: 2.4043197631835938

epho : 0 doc175 batch_num:3 loss:

epho : 0 doc310 batch_num:2 loss: 2.5872533321380615

epho : 0 doc311 batch_num:1 loss: 2.3219892978668213

epho : 0 doc312 batch_num:2 loss: 2.3653604984283447

epho : 0 doc313 batch_num:1 loss: 2.486152172088623

epho : 0 doc314 batch_num:1 loss: 2.588118553161621

epho : 0 doc315 batch_num:1 loss: 2.6141107082366943

epho : 0 doc316 batch_num:1 loss: 2.443807363510132

epho : 0 doc317 batch_num:1 loss: 2.4303297996520996

epho : 0 doc318 batch_num:1 loss: 2.499901056289673

epho : 0 doc319 batch_num:2 loss: 2.471040964126587

epho : 0 doc320 batch_num:1 loss: 2.5685176849365234

epho : 0 doc321 batch_num:1 loss: 2.448167085647583

epho : 0 doc322 batch_num:1 loss: 2.2641396522521973

epho : 0 doc323 batch_num:10 loss: 2.425349235534668

epho : 0 doc324 batch_num:1 loss: 2.4267373085021973

epho : 0 doc325 batch_num:1 loss: 2.476288318634033

epho : 0 doc326 batch_num:3 loss: 2.389695405960083

epho : 0 doc327 batch_num:1 loss: 2.530128240585327

epho : 0 doc328 batch_num:1 loss: 2.3

epho : 0 doc464 batch_num:5 loss: 2.4519054889678955

epho : 0 doc465 batch_num:1 loss: 2.458861827850342

epho : 0 doc466 batch_num:1 loss: 2.4276864528656006

epho : 0 doc467 batch_num:1 loss: 2.627418279647827

epho : 0 doc468 batch_num:1 loss: 2.3786368370056152

epho : 0 doc469 batch_num:1 loss: 2.6246063709259033

epho : 0 doc470 batch_num:1 loss: 2.495481491088867

epho : 0 doc471 batch_num:2 loss: 2.4933223724365234

epho : 0 doc472 batch_num:1 loss: 2.6196603775024414

epho : 0 doc473 batch_num:1 loss: 2.5685815811157227

epho : 0 doc474 batch_num:1 loss: 2.523347854614258

epho : 0 doc475 batch_num:2 loss: 2.4664642810821533

epho : 0 doc476 batch_num:1 loss: 2.372957706451416

epho : 0 doc477 batch_num:3 loss: 2.411421775817871

epho : 0 doc478 batch_num:1 loss: 2.5813775062561035

epho : 0 doc479 batch_num:1 loss: 2.5509114265441895

epho : 0 doc480 batch_num:1 loss: 2.580720901489258

epho : 0 doc481 batch_num:3 loss: 2.450129270553589

epho : 0 doc482 batch_num:2 loss: 2.

epho : 1 doc120 batch_num:18 loss: 2.3512635231018066

epho : 1 doc121 batch_num:1 loss: 2.530033826828003

epho : 1 doc122 batch_num:1 loss: 2.477076292037964

epho : 1 doc123 batch_num:1 loss: 2.4885365962982178

epho : 1 doc124 batch_num:1 loss: 2.301243782043457

epho : 1 doc125 batch_num:1 loss: 2.4074008464813232

epho : 1 doc126 batch_num:2 loss: 2.306535243988037

epho : 1 doc127 batch_num:1 loss: 2.573054552078247

epho : 1 doc128 batch_num:1 loss: 2.237257719039917

epho : 1 doc129 batch_num:1 loss: 2.5350191593170166

epho : 1 doc130 batch_num:2 loss: 2.5319411754608154

epho : 1 doc131 batch_num:1 loss: 2.5978362560272217

epho : 1 doc132 batch_num:1 loss: 2.584890842437744

epho : 1 doc133 batch_num:1 loss: 2.381502866744995

epho : 1 doc134 batch_num:2 loss: 2.5355424880981445

epho : 1 doc135 batch_num:1 loss: 2.3531668186187744

epho : 1 doc136 batch_num:3 loss: 2.516232490539551

epho : 1 doc137 batch_num:1 loss: 2.4923019409179688

epho : 1 doc138 batch_num:5 loss: 2.

epho : 1 doc274 batch_num:2 loss: 2.367081642150879

epho : 1 doc275 batch_num:1 loss: 2.4945790767669678

epho : 1 doc276 batch_num:1 loss: 2.4091827869415283

epho : 1 doc277 batch_num:1 loss: 2.5719876289367676

epho : 1 doc278 batch_num:1 loss: 2.3854236602783203

epho : 1 doc279 batch_num:1 loss: 2.5593645572662354

epho : 1 doc280 batch_num:2 loss: 2.3968420028686523

epho : 1 doc281 batch_num:1 loss: 2.3701822757720947

epho : 1 doc282 batch_num:1 loss: 2.3681044578552246

epho : 1 doc283 batch_num:5 loss: 2.2885334491729736

epho : 1 doc284 batch_num:6 loss: 2.4070043563842773

epho : 1 doc285 batch_num:1 loss: 2.532064914703369

epho : 1 doc286 batch_num:2 loss: 2.4764564037323

epho : 1 doc287 batch_num:2 loss: 2.317227840423584

epho : 1 doc288 batch_num:1 loss: 2.4274935722351074

epho : 1 doc289 batch_num:2 loss: 2.4342434406280518

epho : 1 doc290 batch_num:1 loss: 1.9932178258895874

epho : 1 doc291 batch_num:4 loss: 2.379857063293457

epho : 1 doc292 batch_num:1 loss: 2

epho : 1 doc427 batch_num:1 loss: 2.5069334506988525

epho : 1 doc428 batch_num:2 loss: 2.277298927307129

epho : 1 doc429 batch_num:1 loss: 2.122978925704956

epho : 1 doc430 batch_num:1 loss: 2.5706112384796143

epho : 1 doc431 batch_num:1 loss: 2.5181806087493896

epho : 1 doc432 batch_num:2 loss: 2.3283965587615967

epho : 1 doc433 batch_num:1 loss: 2.5016121864318848

epho : 1 doc434 batch_num:1 loss: 2.091278076171875

epho : 1 doc435 batch_num:10 loss: 2.3863658905029297

epho : 1 doc436 batch_num:1 loss: 2.4378154277801514

epho : 1 doc437 batch_num:1 loss: 2.484365940093994

epho : 1 doc438 batch_num:3 loss: 2.4075162410736084

epho : 1 doc439 batch_num:2 loss: 2.4763455390930176

epho : 1 doc440 batch_num:1 loss: 2.4956765174865723

epho : 1 doc441 batch_num:1 loss: 2.5521416664123535

epho : 1 doc442 batch_num:1 loss: 2.5079128742218018

epho : 1 doc443 batch_num:1 loss: 2.4503650665283203

epho : 1 doc444 batch_num:2 loss: 2.474971294403076

epho : 1 doc445 batch_num:1 loss

epho : 2 doc82 batch_num:1 loss: 2.498206377029419

epho : 2 doc83 batch_num:1 loss: 2.5045723915100098

epho : 2 doc84 batch_num:2 loss: 2.352560520172119

epho : 2 doc85 batch_num:1 loss: 2.474909782409668

epho : 2 doc86 batch_num:2 loss: 2.336968421936035

epho : 2 doc87 batch_num:1 loss: 2.2745275497436523

epho : 2 doc88 batch_num:1 loss: 2.451490879058838

epho : 2 doc89 batch_num:1 loss: 2.298584222793579

epho : 2 doc90 batch_num:2 loss: 2.2625010013580322

epho : 2 doc91 batch_num:1 loss: 2.675640821456909

epho : 2 doc92 batch_num:1 loss: 2.5977699756622314

epho : 2 doc93 batch_num:1 loss: 2.577582597732544

epho : 2 doc94 batch_num:3 loss: 2.427412986755371

epho : 2 doc95 batch_num:2 loss: 2.5379152297973633

epho : 2 doc96 batch_num:1 loss: 2.5088417530059814

epho : 2 doc97 batch_num:1 loss: 2.5164475440979004

epho : 2 doc98 batch_num:2 loss: 2.5129613876342773

epho : 2 doc99 batch_num:1 loss: 2.3113884925842285

epho : 2 doc100 batch_num:1 loss: 2.4421238899230957

e

epho : 2 doc237 batch_num:2 loss: 2.3991315364837646

epho : 2 doc238 batch_num:1 loss: 2.480208158493042

epho : 2 doc239 batch_num:1 loss: 2.5029146671295166

epho : 2 doc240 batch_num:2 loss: 2.299652099609375

epho : 2 doc241 batch_num:1 loss: 2.4505438804626465

epho : 2 doc242 batch_num:1 loss: 2.635099411010742

epho : 2 doc243 batch_num:1 loss: 2.399041175842285

epho : 2 doc244 batch_num:1 loss: 2.2426156997680664

epho : 2 doc245 batch_num:2 loss: 2.472839593887329

epho : 2 doc246 batch_num:1 loss: 2.5012073516845703

epho : 2 doc247 batch_num:2 loss: 2.5108566284179688

epho : 2 doc248 batch_num:3 loss: 2.416574001312256

epho : 2 doc249 batch_num:2 loss: 2.400317907333374

epho : 2 doc250 batch_num:1 loss: 2.356451988220215

epho : 2 doc251 batch_num:2 loss: 2.2327682971954346

epho : 2 doc252 batch_num:1 loss: 2.413508176803589

epho : 2 doc253 batch_num:1 loss: 2.485351800918579

epho : 2 doc254 batch_num:1 loss: 2.499917984008789

epho : 2 doc255 batch_num:1 loss: 2.485

KeyboardInterrupt: 

In [97]:
def top_sim(word,embeddings,top_k):
    index=word2idx[word]
    emb_u=embeddings[index]

    
    sim=np.zeros(len(embeddings))
    for i in range(len(embeddings)):
        sim[i]=(np.dot(emb_u,embeddings[i]))/(np.linalg.norm(emb_u)*np.linalg.norm(embeddings[i]))
    nearest=(-sim).argsort()[1:top_k]
    top_list = []
    for k in range(top_k-1):
        close_word = idx2word[nearest[k]]
        top_list.append(close_word)
    return top_list
top_sim('bank',dcwte.model.get_embeddings(),10)


['deutsche',
 'barclays',
 'rbs',
 'hsbc',
 'lloyds',
 'bank’s',
 'scotland',
 'branches',
 'sale']

In [80]:
print (word2idx)

{'barclays': 0, 'fines': 1, 'ways': 2, 'crisis': 3, 'think': 4, 'bank': 5, 'asked': 6, 'department': 7, 'justice': 8, 'pay': 9, 'large': 10, 'bill': 11, 'financial': 12, 'system': 13, 'mortgage': 14, 'learn': 15, 'view': 16, 'chief': 17, 'executive': 18, 'thinks': 19, 'claims': 20, 'customers': 21, 'clients': 22, 'employees': 23, 'defend': 24, 'against': 25, 'allegations': 26, 'stance': 27, 'possibly': 28, 'going': 29, 'open': 30, 'legal': 31, 'battle': 32, 'powerful': 33, 'especially': 34, 'losing': 35, 'respect': 36, 'order': 37, 'banks': 38, 'approach': 39, 'behind': 40, 'closed': 41, 'difference': 42, 'size': 43, 'demands': 44, 'appear': 45, 'deutsche': 46, 'initially': 47, 'reached': 48, 'settlement': 49, 'thursday': 50, 'reason': 51, 'strong': 52, 'morgan': 53, 'america': 54, 'front': 55, 'obvious': 56, 'created': 57, 'far': 58, 'fewer': 59, 'shouldn’t': 60, 'fine': 61, 'smaller': 62, 'talking': 63, 'hard': 64, 'numbers': 65, 'analysis': 66, 'fair': 67, 'gap': 68, 'wide': 69, 'ta

In [98]:
dcwte.train()

0
epho : 0 doc0 batch_num:1 loss: 2.4277663230895996

epho : 0 doc1 batch_num:1 loss: 2.2080323696136475

epho : 0 doc2 batch_num:1 loss: 2.431504249572754

epho : 0 doc3 batch_num:2 loss: 2.26367449760437

epho : 0 doc4 batch_num:2 loss: 2.365142583847046

epho : 0 doc5 batch_num:1 loss: 2.3160250186920166

epho : 0 doc6 batch_num:1 loss: 2.294538736343384

epho : 0 doc7 batch_num:2 loss: 2.383241653442383

epho : 0 doc8 batch_num:1 loss: 2.452080011367798

epho : 0 doc9 batch_num:1 loss: 2.349925994873047

epho : 0 doc10 batch_num:7 loss: 2.3375444412231445

epho : 0 doc11 batch_num:1 loss: 2.4244863986968994

epho : 0 doc12 batch_num:2 loss: 2.526399612426758

epho : 0 doc13 batch_num:1 loss: 2.397202730178833

epho : 0 doc14 batch_num:2 loss: 2.6219964027404785

epho : 0 doc15 batch_num:6 loss: 2.3234238624572754

epho : 0 doc16 batch_num:2 loss: 2.1846184730529785

epho : 0 doc17 batch_num:1 loss: 2.4298999309539795

epho : 0 doc18 batch_num:1 loss: 2.2211272716522217

epho : 0 do

epho : 0 doc157 batch_num:2 loss: 2.512791156768799

epho : 0 doc158 batch_num:1 loss: 2.4266252517700195

epho : 0 doc159 batch_num:1 loss: 2.487020492553711

epho : 0 doc160 batch_num:2 loss: 2.392810821533203

epho : 0 doc161 batch_num:1 loss: 2.4877045154571533

epho : 0 doc162 batch_num:1 loss: 2.1963329315185547

epho : 0 doc163 batch_num:1 loss: 1.6650118827819824

epho : 0 doc164 batch_num:1 loss: 2.410926103591919

epho : 0 doc165 batch_num:1 loss: 2.058410167694092

epho : 0 doc166 batch_num:1 loss: 2.4150702953338623

epho : 0 doc167 batch_num:2 loss: 2.393402576446533

epho : 0 doc168 batch_num:1 loss: 2.407620668411255

epho : 0 doc169 batch_num:2 loss: 2.355929374694824

epho : 0 doc170 batch_num:1 loss: 2.472268581390381

epho : 0 doc171 batch_num:1 loss: 2.5057287216186523

epho : 0 doc172 batch_num:1 loss: 2.411783456802368

epho : 0 doc173 batch_num:1 loss: 2.314763307571411

epho : 0 doc174 batch_num:1 loss: 2.321922540664673

epho : 0 doc175 batch_num:3 loss: 2.3474

KeyboardInterrupt: 